In [1]:
# HIDE CODE
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import cm
matplotlib.rcParams.update({'font.size': 12})


import warnings
import sys
import os
sys.path.append('/home/server/gli-data-science/')
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses

import os
import ds_db
import helper_db
from helper import transform_to_rupiah, rupiah_format

import pickle
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.1f' % x)
pd.options.mode.chained_assignment = None  # default='warn'
from IPython.display import display, HTML, display_html, IFrame
import ipywidgets as ipyw

def side_by_side_display(dfs:list, captions:list):
    output = ""
    combined = dict(zip(captions, dfs))
    styles = [
        dict(selector="caption", props=[("caption-side", "center"), ("font-size", "100%"), ("color", )])]
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline; font-size:85%' ").set_precision(2).set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0\xa0"

    display(HTML(output))

    
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RANSACRegressor, RidgeClassifier, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeRegressor, export_text, DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVR, SVC, LinearSVC
import xgboost as xgb 


from joblib import dump, load
from sklearn.model_selection import cross_val_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import train_test_split


import textwrap
def split_label(list_label):
    list_label = list(list_label)
    list_label = ["<br>".join(textwrap.wrap(t, width=12)) for t in list_label ]
    return list_label

### ide kasar
> 
* daily job to loop all member alfagift 10 juta
* compute feature trx voucher used, # of trx, # of sales
* compute feature event app site open / view product / atc
* updating each feature and re predict churn feature
* lets play in month - 3 for each define trx and event feature

In [2]:
df_pv_ = pd.read_csv('./feature/pv_31Aug22.csv')
df_pv_['EVENT_TIME'] = pd.to_datetime(df_pv_['EVENT_TIME']).dt.strftime('%Y-%m')

df_pv = df_pv_.groupby(['TRO_PONTA_ID','EVENT_TIME']).agg(PV=('COUNT_VIEW_PRODUCT','sum'))\
        .unstack(level=1).fillna(0).reset_index()

df_pv.columns = [' '.join(col).strip().upper() for col in df_pv.columns.values]

In [3]:
df_re_g_u = pd.read_csv('./feature/rating_2022-09-01.csv')
df_re_g_u = df_re_g_u.rename(columns={'TRO_MEMBERS':'TRO_PONTA_ID'})

In [4]:
# HIDE CODE

## USING ORDER
import glob

df_tto = []
for end_date in pd.date_range('2022-05-01', '2022-09-01', freq='M'):
    start_date = end_date.replace(day=1)
    end_date_save = start_date + relativedelta(months=1)
    
    start_date_str = start_date.strftime('%d%b%y')
    end_date_str = end_date_save.strftime('%d%b%y')
    
    print(start_date_str, end_date_str)
    
    pf = '/home/server/gli-data-science/data/trans_oshop/{}_{}.parquet'.format(start_date_str, end_date_str)
    df_tto.append(pd.read_parquet(pf))
    
df_tto = pd.concat(df_tto)

# df_tto = df_tto[df_tto['TRO_MEMBERS'] != 'N']
df_tto['TRO_DATE'] = pd.to_datetime(df_tto['TRO_DATE'])
df_tto['TRO_NET'] = df_tto['TRO_NET'].astype('float', errors='ignore').astype('int', errors='ignore')


df_tto['TRO_QTY'] = df_tto['TRO_QTY'].astype(int)


## USE THIS FOR GET PV DATA FROM DUMP DATA FILE (SADDDDD ALWAYS DEADLINE)

df_mem_push = df_tto[['TRO_PONTA_ID']].drop_duplicates()
from sqlalchemy import event,create_engine,types


engine_stmt = "oracle://%s:%s@%s/%s" % ('report', 'justd0it', '10.234.152.61', 'alfabi' )
engine = create_engine(engine_stmt)

df = df_mem_push.astype(str)
dtyp = {c:types.VARCHAR(df[c].str.len().max()) for c in df.columns[df.dtypes == 'object'].tolist()}
df.to_sql('temp_churn', engine, index=False, if_exists="replace", dtype=dtyp)

engine.dispose()

01May22 01Jun22
01Jun22 01Jul22
01Jul22 01Aug22
01Aug22 01Sep22


In [5]:
# df_tto['TRO_VOUCHER_USAGE'] = df_tto['TRO_VOUCHER_USAGE']\
#                             .astype('float', errors='ignore').astype('int', errors='ignore')
# df_tto['TRO_DH_SPESIAL'] = df_tto['TRO_DH_SPESIAL']\
#                             .astype('float', errors='ignore').astype('int', errors='ignore')


df_tto_sel = df_tto[(df_tto['TRO_DATE'] >= '2022-05-01') \
            & (df_tto['TRO_DATE'] <= '2022-08-31')]

In [6]:
df_sla = pd.read_csv('./feature/sla_31Aug22.csv')
df_sla['ATT_SEND_DATE_TOSTORE'] = pd.to_datetime(df_sla['ATT_SEND_DATE_TOSTORE'])
df_sla['SLA'] = np.where(df_sla['SLA_NORM']< 0, 20, df_sla['SLA_NORM'])

df_tto_sel = pd.merge(df_tto_sel, df_sla, left_on='TRO_NO_ORDER', right_on='ATT_ORDER_ID', how='left')

col_drop = [
    'ATT_ORDER_ID',
    'ATT_ORDER_DATE',
    'ATT_DELIVERY_DATE',
    'ATT_SEND_DATE_TOSTORE',
    'SLA_NORM'        
]
df_tto_sel = df_tto_sel.drop(col_drop,1)

## create feature
df_tto_sel['SLA'] = df_tto_sel['SLA'].fillna(20)
df_tto_sel['SLA_LATE'] = np.where(df_tto_sel['SLA'] > 60, 1,0)

In [7]:
q = '''
SELECT 
	tc.TRO_PONTA_ID, 
	(TO_DATE('2022/07/08', 'yyyy/mm/dd')) - zt.tgl_registrasi AS SINCE_REGIS
FROM 
	TEMP_CHURN tc
LEFT JOIN (
	select 
		char1 no_member, 
        date2 tgl_registrasi
	from 
		report.ztampungan
	where 
		pk='MEMBERALFAGIFT2'
) zt ON zt.no_member = tc.TRO_PONTA_ID 

'''

con = ds_db.connect_alfabi()

df_regis = pd.read_sql_query(q, con)

con.close()

You are connected


In [8]:
df_regis['SINCE_REGIS'] = df_regis['SINCE_REGIS'].fillna('-1')

In [9]:
df_tto_sel['TRO_DATE_MONTH'] = (df_tto_sel['TRO_DATE'].dt.year.astype(str)
                                + '-0' + df_tto_sel['TRO_DATE'].dt.month.astype(str))

In [10]:
df_tto_sel.head()

,TRO_DATE,TRO_PONTA_ID,TRO_NO_ORDER,TRO_PLU,TRO_QTY,TRO_NET,TRO_DH_SPESIAL,TRO_VOUCHER_CODE,TRO_VOUCHER_USAGE,TRO_PAYMENT_METHOD,TRO_STATUS_RETUR,TRO_SHIP_METHOD_NAME,TRO_KD_STORE,TRO_BRANCH_CODE,SLA,SLA_LATE,TRO_DATE_MONTH
0,2022-05-06,9990013414798400,O-220506-AGVKSHG,103001,5,900,1000.0,BFF3623CEBW2,177700.0,sat-voucher,N,0,NaN,NaN,60.2,1,2022-05
1,2022-05-06,9990013414798400,O-220506-AGVKSHG,192510,1,45855,0.0,BFF3623CEBW2,177700.0,sat-voucher,N,0,NaN,NaN,60.2,1,2022-05
2,2022-05-06,9990013414798400,O-220506-AGVKSHG,211141,1,22432,0.0,BFF3623CEBW2,177700.0,sat-voucher,N,0,NaN,NaN,60.2,1,2022-05
3,2022-05-06,9990013414798400,O-220506-AGVKSHG,402359,1,31900,0.0,BFF3623CEBW2,177700.0,sat-voucher,N,0,NaN,NaN,60.2,1,2022-05
4,2022-05-06,9990013414798400,O-220506-AGVKSHG,414796,2,9009,0.0,BFF3623CEBW2,177700.0,sat-voucher,N,0,NaN,NaN,60.2,1,2022-05


In [11]:
df_tto_sel_g = df_tto_sel.groupby(['TRO_PONTA_ID','TRO_DATE_MONTH','TRO_NO_ORDER'])\
                        .agg({'TRO_NET':'sum',
                              'TRO_NO_ORDER':'nunique',
                              'SLA':'first',
                              'SLA_LATE':'first',
                              'TRO_PAYMENT_METHOD':'first'
                             })
df_tto_sel_g = df_tto_sel_g.groupby(['TRO_PONTA_ID','TRO_DATE_MONTH'])\
                        .agg({'TRO_NET':'sum',
                              'TRO_NO_ORDER':'sum',
                              'SLA':'sum',
                              'SLA_LATE':'sum',
                              'TRO_PAYMENT_METHOD':'nunique'
                             })

In [12]:
df_tto_sel_g.head()

TRO_NET  TRO_NO_ORDER  SLA  SLA_LATE  \
TRO_PONTA_ID     TRO_DATE_MONTH                                         
9990010000001200 2022-05          192099             4 44.2         0   
                 2022-06          264662             6 82.1         0   
                 2022-07          214246             5 85.1         0   
                 2022-08          632338            10 92.9         0   
9990010000001771 2022-05          552242             6 81.5         0   

                                 TRO_PAYMENT_METHOD  
TRO_PONTA_ID     TRO_DATE_MONTH                      
9990010000001200 2022-05                          2  
                 2022-06                          3  
                 2022-07                          2  
                 2022-08                          2  
9990010000001771 2022-05                          2

## unstack all feature in oshop

In [13]:
df_tto_sel_g_u = df_tto_sel_g.unstack(level=1).fillna(0).reset_index()
df_tto_sel_g_u.columns = [' '.join(col).strip().upper() for col in df_tto_sel_g_u.columns.values]

In [14]:
df_re_g_u.head()

,TRO_PONTA_ID,RATING_BELOW 2022-05,RATING_BELOW 2022-06,RATING_BELOW 2022-07,RATING_BELOW 2022-08,RATING_AVG 2022-05,RATING_AVG 2022-06,RATING_AVG 2022-07,RATING_AVG 2022-08
0,9990010000001200,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
1,9990010000090310,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
2,9990010000102923,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
3,9990010000119228,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
4,9990010000131020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0


## merge with rating

In [15]:
df_re_g_u['TRO_PONTA_ID'] = df_re_g_u['TRO_PONTA_ID'].astype(str)
df_tto_re = pd.merge(df_tto_sel_g_u, df_re_g_u, on='TRO_PONTA_ID', how='left').fillna(0)

## merge with product view

In [16]:
df_pv['TRO_PONTA_ID'] = df_pv['TRO_PONTA_ID'].astype(str)
df_tto_re_pv = pd.merge(df_tto_re, df_pv, on='TRO_PONTA_ID', how='left').fillna(0)

## merge with regis

In [17]:
df_tto_re_pv = pd.merge(df_tto_re_pv, df_regis, on='TRO_PONTA_ID')
df_tto_re_pv = df_tto_re_pv[df_tto_re_pv['SINCE_REGIS'].astype(int) > 0]

In [18]:
df_tto_re_pv.head()

,TRO_PONTA_ID,TRO_NET 2022-05,TRO_NET 2022-06,TRO_NET 2022-07,TRO_NET 2022-08,TRO_NO_ORDER 2022-05,TRO_NO_ORDER 2022-06,TRO_NO_ORDER 2022-07,TRO_NO_ORDER 2022-08,SLA 2022-05,SLA 2022-06,SLA 2022-07,SLA 2022-08,SLA_LATE 2022-05,SLA_LATE 2022-06,SLA_LATE 2022-07,SLA_LATE 2022-08,TRO_PAYMENT_METHOD 2022-05,TRO_PAYMENT_METHOD 2022-06,TRO_PAYMENT_METHOD 2022-07,TRO_PAYMENT_METHOD 2022-08,RATING_BELOW 2022-05,RATING_BELOW 2022-06,RATING_BELOW 2022-07,RATING_BELOW 2022-08,RATING_AVG 2022-05,RATING_AVG 2022-06,RATING_AVG 2022-07,RATING_AVG 2022-08,PV 2022-05,PV 2022-06,PV 2022-07,PV 2022-08,SINCE_REGIS
0,9990010000001200,192099.0,264662.0,214246.0,632338.0,4.0,6.0,5.0,10.0,44.2,82.1,85.1,92.9,0.0,0.0,0.0,0.0,2.0,3.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5.0,1.0,3.0,1144.0
1,9990010000001771,552242.0,748387.0,1589562.0,145509.0,6.0,30.0,21.0,2.0,81.5,280.9,309.0,40.0,0.0,0.0,1.0,0.0,2.0,3.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,8.0,2.0,1079.0
2,9990010000001999,0.0,59480.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,1870.8,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,4.0,1.0,1117.0
3,9990010000003113,0.0,195671.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,38.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,972.0
4,9990010000004582,0.0,0.0,0.0,79366.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,21.0,956.0


In [19]:
# df_tto_re_pv['NET 11-10'] = df_tto_re_pv['TRO_NET 2021-11'] - df_tto_re_pv['TRO_NET 2021-10']
# df_tto_re_pv['NET 10-09'] = df_tto_re_pv['TRO_NET 2021-10'] - df_tto_re_pv['TRO_NET 2021-09']
# df_tto_re_pv['NET 11-09'] = df_tto_re_pv['TRO_NET 2021-11'] - df_tto_re_pv['TRO_NET 2021-09']

# df_tto_re_pv['STRUK 11-10'] = df_tto_re_pv['TRO_NO_ORDER 2021-11'] - df_tto_re_pv['TRO_NO_ORDER 2021-10']
# df_tto_re_pv['STRUK 10-09'] = df_tto_re_pv['TRO_NO_ORDER 2021-10'] - df_tto_re_pv['TRO_NO_ORDER 2021-09']
# df_tto_re_pv['STRUK 11-09'] = df_tto_re_pv['TRO_NO_ORDER 2021-11'] - df_tto_re_pv['TRO_NO_ORDER 2021-09']

# df_tto_re_pv['PV 11-10'] = df_tto_re_pv['PV 2021-11'] - df_tto_re_pv['PV 2021-10']
# df_tto_re_pv['PV 10-09'] = df_tto_re_pv['PV 2021-10'] - df_tto_re_pv['PV 2021-09']
# df_tto_re_pv['PV 11-09'] = df_tto_re_pv['PV 2021-11'] - df_tto_re_pv['PV 2021-09']

# df_tto_re_pv['SLA 11-10'] = df_tto_re_pv['SLA 2021-11'] - df_tto_re_pv['SLA 2021-10']
# df_tto_re_pv['SLA 10-09'] = df_tto_re_pv['SLA 2021-10'] - df_tto_re_pv['SLA 2021-09']
# df_tto_re_pv['SLA 11-09'] = df_tto_re_pv['SLA 2021-11'] - df_tto_re_pv['SLA 2021-09']

# df_tto_re_pv['BASKET 11'] = df_tto_re_pv['TRO_NET 2021-11']/df_tto_re_pv['TRO_NO_ORDER 2021-11']
# df_tto_re_pv['BASKET 10'] = df_tto_re_pv['TRO_NET 2021-10']/df_tto_re_pv['TRO_NO_ORDER 2021-10']
# df_tto_re_pv['BASKET 09'] = df_tto_re_pv['TRO_NET 2021-09']/df_tto_re_pv['TRO_NO_ORDER 2021-09']

# df_tto_re_pv['BASKET AVG'] = (df_tto_re_pv['BASKET 11'] + df_tto_re_pv['BASKET 10'] + df_tto_re_pv['BASKET 09']) / 3

In [20]:
# print(df_ins[df_ins['label'] == 1]['NET_11-10'].median())
# print(df_ins[df_ins['label'] == 1]['NET_10-09'].median())
# print(df_ins[df_ins['label'] == 1]['NET_11-09'].median())
# print(df_ins[df_ins['label'] == 1]['STRUK_11-10'].mean())
# print(df_ins[df_ins['label'] == 1]['STRUK_10-09'].mean())
# print(df_ins[df_ins['label'] == 1]['STRUK_11-09'].mean())

## Random Forest

### not oversampling version without threshold 2/3 or 3/3

In [21]:
from sklearn.model_selection import StratifiedKFold
condition_2_3 = (
((df_tto_re_pv['TRO_NET 2022-05'] > 0) & \
(df_tto_re_pv['TRO_NET 2022-06'] > 0) & \
(df_tto_re_pv['TRO_NET 2022-07'] > 0) & \
(df_tto_re_pv['TRO_NET 2022-08'] == 0)) | \
    
((df_tto_re_pv['TRO_NET 2022-05'] > 0) & \
(df_tto_re_pv['TRO_NET 2022-06'] == 0) & \
(df_tto_re_pv['TRO_NET 2022-07'] > 0) & \
(df_tto_re_pv['TRO_NET 2022-08'] == 0)) |
    
((df_tto_re_pv['TRO_NET 2022-05'] == 0) & \
(df_tto_re_pv['TRO_NET 2022-06'] > 0) & \
(df_tto_re_pv['TRO_NET 2022-07'] > 0) & \
(df_tto_re_pv['TRO_NET 2022-08'] == 0))
)

condition_3_3 = ( 
          (df_tto_re_pv['TRO_NET 2022-05'] > 0) & \
          (df_tto_re_pv['TRO_NET 2022-06'] > 0) & \
          (df_tto_re_pv['TRO_NET 2022-07'] > 0) & \
          (df_tto_re_pv['TRO_NET 2022-08'] == 0) 
)

df_tto_re_pv['label'] = np.where(condition_3_3, 1, 0)


X = df_tto_re_pv.drop(
[
'TRO_PONTA_ID',
'TRO_NET 2022-08', 
'TRO_NO_ORDER 2022-08',
'SLA 2022-08',
'SLA_LATE 2022-08',
'RATING_BELOW 2022-08',
'RATING_AVG 2022-08',
'PV 2022-08',
'TRO_PAYMENT_METHOD 2022-08',
'label'
], 1
)
y = df_tto_re_pv[['label']]


# X['NET-1'] = X.iloc[:,2] - X.iloc[:,1]
# X['NET-2'] = X.iloc[:,1] - X.iloc[:,0]


# X['ORDER-1'] = X.iloc[:,8] - X.iloc[:,7]
# X['ORDER-2'] = X.iloc[:,7] - X.iloc[:,6]


# X['SLA-1'] = X.iloc[:,11] - X.iloc[:,10]
# X['SLA-2'] = X.iloc[:,10] - X.iloc[:,9]


# X['PV-1'] = X.iloc[:,29] - X.iloc[:,28]
# X['PV-2'] = X.iloc[:,28] - X.iloc[:,27]


# skf = StratifiedKFold(n_splits=5)
# for train_index, test_index in skf.split(X, y):
#     print("="*50)
#     X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
#     y_train, y_test = y.iloc[train_index,:], y.iloc[test_index,:]


    
#     ## applying standard scaler
#     scaler = StandardScaler()
#     scaler.fit(X_train)
#     X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
#     X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_train.columns)
    

#     #clf_ridge = RandomForestClassifier(random_state=42) 
#     clf_ridge = MLPClassifier(random_state=1, max_iter=300, hidden_layer_sizes=(70,))
#     clf_ridge.fit(X_train, y_train) 
#     pred = clf_ridge.predict(X_test)
#     print(classification_report(y_test, pred))
#     fig, ax = plt.subplots(figsize=(5, 5), dpi=100)
#     ConfusionMatrixDisplay.from_predictions(y_test, pred, 
#                                             cmap='Blues', display_labels= ['not_churn', 'churn'], 
#                                             ax = ax, values_format = 'd')


# applying standard scaler
scaler = StandardScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), index=X.index, columns=X.columns)
dump(scaler, './model/scaler_70_aug22_3_3.joblib')


clf_all_mlp = MLPClassifier(random_state=1, max_iter=300, hidden_layer_sizes=(70,))
clf_all_mlp.fit(X, y) 
dump(clf_all_mlp, './model/mlp_70_aug22_3_3.joblib')




['./model/mlp_70_aug22_3_3.joblib']

In [22]:
# a = pd.DataFrame()

# a['NET 11-10'] = df_tto_re_pv['TRO_NET 2021-12'] - df_tto_re_pv['TRO_NET 2021-11']
# a['NET 10-09'] = df_tto_re_pv['TRO_NET 2021-11'] - df_tto_re_pv['TRO_NET 2021-10']
# a['NET 11-09'] = df_tto_re_pv['TRO_NET 2021-12'] - df_tto_re_pv['TRO_NET 2021-10']

# a['STRUK 11-10'] = df_tto_re_pv['TRO_NO_ORDER 2021-12'] - df_tto_re_pv['TRO_NO_ORDER 2021-11']
# a['STRUK 10-09'] = df_tto_re_pv['TRO_NO_ORDER 2021-11'] - df_tto_re_pv['TRO_NO_ORDER 2021-10']
# a['STRUK 11-09'] = df_tto_re_pv['TRO_NO_ORDER 2021-12'] - df_tto_re_pv['TRO_NO_ORDER 2021-10']

# a['PV 11-10'] = df_tto_re_pv['PV 2021-12'] - df_tto_re_pv['PV 2021-11']
# a['PV 10-09'] = df_tto_re_pv['PV 2021-11'] - df_tto_re_pv['PV 2021-10']
# a['PV 11-09'] = df_tto_re_pv['PV 2021-12'] - df_tto_re_pv['PV 2021-10']

# a['SLA 11-10'] = df_tto_re_pv['SLA 2021-12'] - df_tto_re_pv['SLA 2021-11']
# a['SLA 10-09'] = df_tto_re_pv['SLA 2021-11'] - df_tto_re_pv['SLA 2021-10']
# a['SLA 11-09'] = df_tto_re_pv['SLA 2021-12'] - df_tto_re_pv['SLA 2021-10']

# a['BASKET 11'] = df_tto_re_pv['TRO_NET 2021-12']/df_tto_re_pv['TRO_NO_ORDER 2021-12']
# a['BASKET 10'] = df_tto_re_pv['TRO_NET 2021-11']/df_tto_re_pv['TRO_NO_ORDER 2021-11']
# a['BASKET 09'] = df_tto_re_pv['TRO_NET 2021-10']/df_tto_re_pv['TRO_NO_ORDER 2021-10']

# a['BASKET AVG'] = (df_tto_re_pv['BASKET 11'] + df_tto_re_pv['BASKET 10'] + df_tto_re_pv['BASKET 09']) / 3

In [23]:
X_12 = df_tto_re_pv.drop(
[
'TRO_PONTA_ID',
'TRO_NET 2022-05', 
'TRO_NO_ORDER 2022-05',
'SLA 2022-05',
'SLA_LATE 2022-05',
'RATING_BELOW 2022-05',
'RATING_AVG 2022-05',
'PV 2022-05',
'TRO_PAYMENT_METHOD 2022-05',
'label'
], 1
)
X_12['SINCE_REGIS'] = X_12['SINCE_REGIS']+30

# X_12 = pd.concat([X_12.iloc[:,0:-16],a],1)
# X_12 = X_12.fillna(0)
# y_12 = df_tto_re_pv['label']

In [24]:
X_12.shape

(1249135, 25)

In [25]:
# X_12.head()

In [26]:


scaler_all = load('./model/scaler_70_aug22_3_3.joblib')
clf_all = load('./model/mlp_70_aug22_3_3.joblib')


X_12_scale = pd.DataFrame(scaler_all.transform(X_12), index=X_12.index, columns=X_12.columns)
pred = (clf_all.predict_proba(X_12_scale)[:,1] >= 0.55).astype(int)


df_mem_12 = pd.concat([df_tto_re_pv[['TRO_PONTA_ID']].reset_index(drop=True)
                       , pd.DataFrame(pred, columns=['PRED'])], 1)

In [27]:
print("PREDICTION RESULT {}".format(df_mem_12[df_mem_12['PRED'] == 1].shape))

PREDICTION RESULT (10220, 2)


In [28]:
df_trader = pd.read_excel('/home/server/gli-data-science/ferina/Sales Trader Ytd Aug-ExEmployee.xlsx'
                          , dtype='object', names=['member','stat'])

In [29]:
df_mem_12 = df_mem_12[~df_mem_12['TRO_PONTA_ID'].isin(df_trader['member'])]

In [30]:
print("AFTER REMOVE TRADER {}".format(df_mem_12[df_mem_12['PRED'] == 1].shape))

AFTER REMOVE TRADER (10174, 2)


In [31]:
df.head()

,TRO_PONTA_ID
0,9990013414798400
6,9990012462424798
10,9990011438031753
17,9990013662549600
22,9990010631498000


In [32]:
engine = create_engine(engine_stmt)

df = df_mem_12[df_mem_12['PRED'] == 1].astype(str)
dtyp = {c:types.VARCHAR(df[c].str.len().max()) for c in df.columns[df.dtypes == 'object'].tolist()}
df.to_sql('temp_churn_target', engine, index=False, if_exists="replace", dtype=dtyp)

engine.dispose()

In [34]:
q = '''
SELECT DISTINCT(tct.TRO_PONTA_ID)
FROM TEMP_CHURN_TARGET tct
WHERE 
tct.TRO_PONTA_ID NOT IN (
	SELECT 
		gua.GUA_ACCOUNT_CARD_ADJ
	FROM GLI_UNINSTALL_ALFAGIFT gua 
	WHERE 
		gua.GUA_STATUS = 'Y'
		AND trunc(GUA_UNINSTALL_DATE) BETWEEN '01-aug-21' AND '30-sep-22'
)


'''

con = ds_db.connect_alfabi()
df_churn_sel = pd.read_sql_query(q, con)
con.close()


df_exclude = pd.read_parquet(
    '/home/server/gli-data-science/data/trans_oshop/01Sep22_01Oct22.parquet'
)
df_churn_sel = df_churn_sel[~df_churn_sel['TRO_PONTA_ID'].isin(df_exclude['TRO_PONTA_ID'])]

You are connected


In [35]:
df_churn_sel.shape

(6887, 1)

In [36]:
df_churn_sel['PERIOD'] = '2022-09-01'
df_churn_sel['PERIOD'] = pd.to_datetime(df_churn_sel['PERIOD'])
df_churn_sel = df_churn_sel[['PERIOD','TRO_PONTA_ID']].rename(columns={'TRO_PONTA_ID':'MEMBER'})
df_churn_all = df_churn_sel.copy()

In [37]:
df_churn_all.shape

(6887, 2)

In [38]:


from sqlalchemy import event,create_engine,types



driver = 'cx_oracle'
server = '10.234.152.61' 
database = 'alfabi' 
username = 'report' 
password = 'justd0it'
engine_stmt = "oracle://%s:%s@%s/%s" % (username, password, server, database )
    
engine = create_engine(engine_stmt)

q = '''

DELETE FROM ALFAGIFT_CHURN
WHERE PERIOD = '1-sep-22'

'''

con = engine.connect()
con.execute(q)

df = df_churn_all.copy()
dtyp = {c:types.VARCHAR(df[c].str.len().max()) for c in df.columns[df.dtypes == 'object'].tolist()}
df.to_sql('ALFAGIFT_CHURN', engine, index=False, if_exists="append", dtype=dtyp)



engine.dispose()

In [39]:
df_churn_all.head(20)

,PERIOD,MEMBER
2,2022-09-01,9990012163418482
3,2022-09-01,9990013593089629
5,2022-09-01,9990013873150693
7,2022-09-01,9990013315653485
8,2022-09-01,9990013324357517
9,2022-09-01,9990013860033544
10,2022-09-01,9990013578177634
11,2022-09-01,9990013859188800
12,2022-09-01,9990013863548378
13,2022-09-01,9990017006608153


In [41]:
df_churn_view = pd.merge(df_churn_sel,df_tto_re_pv,left_on='MEMBER'
                         ,right_on='TRO_PONTA_ID', how='left')

In [42]:
df_churn_view.head(20)

,PERIOD,MEMBER,TRO_PONTA_ID,TRO_NET 2022-05,TRO_NET 2022-06,TRO_NET 2022-07,TRO_NET 2022-08,TRO_NO_ORDER 2022-05,TRO_NO_ORDER 2022-06,TRO_NO_ORDER 2022-07,TRO_NO_ORDER 2022-08,SLA 2022-05,SLA 2022-06,SLA 2022-07,SLA 2022-08,SLA_LATE 2022-05,SLA_LATE 2022-06,SLA_LATE 2022-07,SLA_LATE 2022-08,TRO_PAYMENT_METHOD 2022-05,TRO_PAYMENT_METHOD 2022-06,TRO_PAYMENT_METHOD 2022-07,TRO_PAYMENT_METHOD 2022-08,RATING_BELOW 2022-05,RATING_BELOW 2022-06,RATING_BELOW 2022-07,RATING_BELOW 2022-08,RATING_AVG 2022-05,RATING_AVG 2022-06,RATING_AVG 2022-07,RATING_AVG 2022-08,PV 2022-05,PV 2022-06,PV 2022-07,PV 2022-08,SINCE_REGIS,label
0,2022-09-01,9990012163418482,9990012163418482,0.0,48734.0,133860.0,12881.0,0.0,4.0,11.0,1.0,0.0,132.6,284.4,31.3,0.0,1.0,0.0,0.0,0.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,16.0,1.0,958.0,0
1,2022-09-01,9990013593089629,9990013593089629,0.0,437179.0,42609.0,20180.0,0.0,32.0,3.0,1.0,0.0,80725.9,778.1,20.0,0.0,15.0,3.0,0.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,24.0,16.0,23.0,161.0,0
2,2022-09-01,9990013873150693,9990013873150693,0.0,260626.0,336302.0,13680.0,0.0,3.0,5.0,1.0,0.0,7050.4,439.5,83.7,0.0,1.0,1.0,1.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,12.0,0
3,2022-09-01,9990013315653485,9990013315653485,121626.0,182967.0,57923.0,15675.0,4.0,12.0,5.0,1.0,21.4,19554.0,5255.5,20.0,0.0,8.0,3.0,0.0,1.0,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,10.0,7.0,4.0,442.0,0
4,2022-09-01,9990013324357517,9990013324357517,155395.0,461505.0,139422.0,7927.0,9.0,48.0,7.0,2.0,9312.9,116258.4,3716.8,40.0,4.0,34.0,4.0,0.0,5.0,5.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,56.0,71.0,10.0,431.0,0
5,2022-09-01,9990013860033544,9990013860033544,0.0,209637.0,368806.0,13950.0,0.0,3.0,6.0,1.0,0.0,60.0,5060.1,8.7,0.0,0.0,3.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19.0,0
6,2022-09-01,9990013578177634,9990013578177634,200395.0,150332.0,15765.0,6035.0,2.0,2.0,1.0,1.0,27.3,7.6,167.2,1131.6,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,165.0,0
7,2022-09-01,9990013859188800,9990013859188800,0.0,306933.0,22972.0,16125.0,0.0,3.0,1.0,1.0,0.0,659.5,2.0,20.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,19.0,0
8,2022-09-01,9990013863548378,9990013863548378,0.0,144142.0,95314.0,10358.0,0.0,2.0,1.0,1.0,0.0,70.8,1.8,1930.4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,17.0,0
9,2022-09-01,9990017006608153,9990017006608153,0.0,29277.0,13963.0,17566.0,0.0,3.0,1.0,1.0,0.0,13853.0,355.9,1339.5,0.0,3.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,2.0,3.0,1060.0,0
